First load the training data

In [ ]:
import pickle
import numpy as np

train_input = pickle.load(open('Train_input', 'rb'))
train_output = pickle.load(open( 'Train_output', 'rb'))

Lets add [start] and [end] words to the output sequences

In [ ]:
input_pairs = []
for i, txt in enumerate(train_output):
  txt = "[start] " + txt + "[end]"
  input_pairs.append((train_input[i], txt))

print(len(input_pairs))

112000


Lets check the vocabulary size...

In [ ]:
import random

input_words = set()
for txt, _ in input_pairs:
  for word in txt.split(" "):
    if word not in input_words:
      input_words.add(word)

print(len(input_words))
print(input_words)

for _, txt in input_pairs:
  for word in txt.split(" "):
    if word not in input_words:
      input_words.add(word)
print(len(input_words))
print(input_words)

output_words = set()
for _, txt in input_pairs:
  for word in txt.split(" "):
    if word not in output_words:
      output_words.add(word)
print(len(output_words))
print(output_words)

print(random.choice(input_pairs))

9
{'', 'g', 'a', 'f', 'b', 'h', 'd', 'e', 'c'}
21
{'', 'g', 'a', 'ee', 'm', '[end]', 'ef', 'd', 'eh', 'k', 'f', 'ed', 'b', 'l', 'h', 'i', 'c', 'eg', 'e', '[start]', 'j'}
20
{'g', 'a', 'ee', 'm', '[end]', 'ef', 'd', 'eh', 'k', 'f', 'ed', 'b', 'l', 'h', 'i', 'c', 'eg', 'e', '[start]', 'j'}
('a d a d a e a d b d a d b d c f c f c g ', '[start] b d b d c f a d e f a d d g a e h c f a d i j c g a d k l [end]')


There are 21 combined words from the 2 datasets, so ill just use vocab size 25 to be safe

In [ ]:
from matplotlib import pyplot as plt

input_lengths = [len(txt.split(" ")) for txt in train_input]
output_lengths = [len(txt.split(" ")) for txt in train_output]

print(max(len(txt.split(" ")) for txt in train_input))
print(max(len(txt.split(" ")) for txt in train_output))

33
48


The max length of the sequences is 48, so ill use 48 as my sequence length.

Now we can use textvectorization to prepare the training data

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers
import string
import re

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

input_sequence_length = 48
output_sequence_length = 48

input_vocab_size = 25
output_vocab_size = 25

sequence_length = 48
vocab_size = 25

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_source_texts = [pair[0] for pair in input_pairs]
train_target_texts = [pair[1] for pair in input_pairs]
source_vectorization.adapt(train_source_texts)
target_vectorization.adapt(train_target_texts)

Partition the data into training, validation, and test datasets

In [ ]:
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences

def format_dataset(source_text, target_text):
  source = source_vectorization(source_text)
  target = target_vectorization(target_text)

  return ({ 'source' : source, 'target' : target[:, :-1] }, target[:, 1:])

def make_dataset(input_pairs):
  source_texts, target_texts = zip(*input_pairs)
  source_texts = list(source_texts)
  target_texts = list(target_texts)
  dataset = tf.data.Dataset.from_tensor_slices((source_texts, target_texts))
  dataset = dataset.batch(256)
  dataset = dataset.map(format_dataset, num_parallel_calls=4)
  return dataset.shuffle(256).prefetch(16).cache()

random.Random(2048).shuffle(input_pairs)
num_train_sequences = int(0.7 * len(input_pairs))
num_validate_sequences = int(0.15 * len(input_pairs))

train_dataset = make_dataset(input_pairs[:num_train_sequences])
validation_dataset = make_dataset(input_pairs[num_train_sequences:num_train_sequences + num_validate_sequences])
test_dataset = make_dataset(input_pairs[num_train_sequences + num_validate_sequences:])

In [ ]:
for inputs, targets in train_dataset.take(1):
  print(f'inputs["source"] shape: {inputs["source"].shape}')
  print(f'inputs["target"] shape: {inputs["target"].shape}')
  print(f'targets.shape: {targets.shape}')

inputs["source"] shape: (256, 48)
inputs["target"] shape: (256, 48)
targets.shape: (256, 48)


For the model I will use the PositionalEmbedding, TransformerEncoder, and TransformerDecoder layers provided by the textbook.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


@keras.saving.register_keras_serializable(package="MyLayers")
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

@keras.saving.register_keras_serializable(package="MyLayers")
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

@keras.saving.register_keras_serializable(package="MyLayers")
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

I tried using 2048 for the dense dimmension but I changed that to 1024 instead as that got better performance

In [ ]:
embed_dim = 256
dense_dim = 1024
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="source")
x = PositionalEmbedding(input_sequence_length, input_vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="target")
x = PositionalEmbedding(output_sequence_length, output_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(output_sequence_length, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Bellow you can see the training process for the model

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

save_model = keras.callbacks.ModelCheckpoint("Machine_Translation_Model.h5", save_best_only=True, monitor="val_loss")

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

transformer.fit(train_dataset, epochs=100, validation_data=validation_dataset, callbacks=[early_stopping, save_model])

Epoch 1/100
307/307 [==============================] - 86s 264ms/step - loss: 1.6281 - accuracy: 0.4610 - val_loss: 0.8840 - val_accuracy: 0.6371
Epoch 2/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


307/307 [==============================] - 80s 260ms/step - loss: 0.8712 - accuracy: 0.6481 - val_loss: 0.6379 - val_accuracy: 0.7317
Epoch 3/100
307/307 [==============================] - 80s 260ms/step - loss: 0.6683 - accuracy: 0.7283 - val_loss: 0.5633 - val_accuracy: 0.7598
Epoch 4/100
307/307 [==============================] - 80s 261ms/step - loss: 0.5572 - accuracy: 0.7694 - val_loss: 0.4613 - val_accuracy: 0.8043
Epoch 5/100
307/307 [==============================] - 80s 261ms/step - loss: 0.4932 - accuracy: 0.7932 - val_loss: 0.4520 - val_accuracy: 0.8058
Epoch 6/100
307/307 [==============================] - 80s 260ms/step - loss: 0.4445 - accuracy: 0.8108 - val_loss: 0.3983 - val_accuracy: 0.8250
Epoch 7/100
307/307 [==============================] - 80s 260ms/step - loss: 0.4011 - accuracy: 0.8260 - val_loss: 0.4019 - val_accuracy: 0.8250
Epoch 8/100
307/307 [==============================] - 80s 260ms/step - loss: 0.3783 - accuracy: 0.8353 - val_loss: 0.3789 - val_accurac

So we can see the model is working very well and we are able to predict the next token with 99.76% validation accuracy